# Constraint scan

## Generate constraintscandf

In [ ]:
import numpy as np
import pandas as pd
import pyDOE
from mmon_gcm.constraintscan import generate_constraints_df

## Define parameter bounds

In [ ]:
parameter_bounds = {}

## Photons

We can go 10% above and below the value for photon reflectance and transmission from Zhu et al (2010)

In [ ]:
parameter_bounds["P_abs"] = [0.9*0.9, 0.9*1.1, r"Dimensionless",
                             "90% of Zhu, Long, and Ort (2010)", "110% of Zhu, Long, and Ort (2010)"]  # proportion of photons absorbed by the leaf

The lower bound is the default we've used from Wuyts et al (2021), for a higher value we'll use one from Ramonell 2001.

In [ ]:
parameter_bounds["T_l"] = [0.017 * 10 ** -2, 0.024 * 10 ** -2, r"m", "Wuyts et al. (2010)", "Ramonell et al. (2001)"]

The are of the leaf being simulated is fixed at 1m$^2$

In [ ]:
parameter_bounds["A_l"] = [1, 1, r"m$^2$", "Fixed", "Fixed"]

For the lower bound we can use the average volume of an *Arabidopsis* guard cell as the midpoint of the values given in table 1 of Jezek and Blatt (2017). For the upper bound we can use the volume given in Hills et al (2012). 

In [ ]:
j_b_upper = 0.65
j_b_lower = 0.3
V_gc_ind = (j_b_lower+j_b_upper)/2  # pL
V_gc_ind = V_gc_ind * 10**-12  # dm3
parameter_bounds["V_gc_ind"] = [V_gc_ind, 4.1E-12, r"dm$^3$", "Jezek and Blatt (2017)", "Hills et al. (2012)"]

The photosynthetic efficient of guard cells compared to the mesophyll was taken from Lawson (2003). 

> In leaves of all species the values of photosynthetic efficiency for guard cells were either indistinguishable from or only slightly lower (minimum of 79%) than those of the underlying, spongy mesophyll cells.

We'll use these as the bounds.

In [ ]:
parameter_bounds["FqFm"] = [0.79, 0.9, r"Dimensionless", "Lawson (2003)", "Lawson (2003)"]

Fujiwara et al (2019) provide a range of values for number of chloroplasts in mesophyll and guard cells, we can take the upper gc and lower me and vice versa to get the upper and lower bound for the ratios, respectively.

In [ ]:
upper_gc = 5.5
lower_gc = 3.5

upper_me = 100
lower_me = 30

rch_lower_bound = lower_gc/upper_me
rch_upper_bound = upper_gc/lower_me

parameter_bounds["R_ch"] = [rch_lower_bound, rch_upper_bound, r"Dimensionless",
                            "Fujiwara, Sanjaya, and Itoh (2019)", "Fujiwara, Sanjaya, and Itoh (2019)"]

For the lower bound Ramonell et al. (2001) provide a percentage of the leaf that is space at atmospheric pressure in Table 1.For the upper bound we take the proportion of the leaf that is air from Earles et al 2018 for *Guzmania zahnii* from Table 2. 

In [ ]:
parameter_bounds["L_air"] = [0.185, 0.37, r"Dimensionless", "Ramonell et al. (2001)", "Earles et al. (2018)"]

Willmer and Fricker (1996) provide a lower bound of 0.1 and Ramonell et al. (2001) an upper bound of 0.24 for the proportion of the leaf that is epidermis.

In [ ]:
parameter_bounds["L_epidermis"] = [0.1, 0.24, r"Dimensionless", "Willmer and Fricker (1996)", "Ramonell et al. (2001)"]

## Osmolarity

Wang et al. (2017) use a value of 0.751 to be the proportion of the Guard Cell that is vacuole, Andrés et al. (2014) quote a value of 90% from an old MacRobbie paper.  

In [ ]:
parameter_bounds["Vac_frac"] = [0.751, 0.9, r"Dimensionless", "Wang et al. (2017)", "Andrés et al. (2014) "]

A reasonable range for temperature seems to be 10C to 25C, this covers a wide range:

In [ ]:
parameter_bounds["T"] = [273.15+10, 273.15+25, "K", "10C", "25C"]

Ideal gas constant, from NIST, (reference Tiesinga2019). This is constant, so won't change in the model

In [ ]:
R = 8.205*10**(-5)  # m3atmK-1mol-1
R = R*10**3  # dm3atmK-1mol-1
parameter_bounds["R"] = [
    R, R, r"dm$^3$$\cdot$atm$\cdot$K$^{-1}$$\cdot$mol$^{-1}$", "Tiesinga et al. (2019)", "Tiesinga et al. (2019)"]

The density of guard cells per m$^2$ of leaf was taken by multiplying the density of stomata from Papanatsiou et al (2016) by two. This only takes into account the abaxial surface of the leaf, so for the upper bound, we'll double this again. For the lower bound, we'll use a value for *Commelina* from Willmer and Fricker (1996). 

In [ ]:
parameter_bounds["N_gcs"] = [172 * 10 ** 6, 290 * 10 ** 6 * 2.0 * 2,
                             r"GCs$\cdot$m$^{-2}$", "Willmer and Fricker (1996)", "Papanatsiou, Amtmann, and Blatt (2016)"]

The following parameters required for osmolarity are based taking values from an older OnGuard paper (Wang et al. (2012)) and the newer, updated model (Wang et al 2017).

In [ ]:
parameter_bounds["n"] = [1.5, 2.5, "atm", "Wang et al. (2012)", "Wang et al. (2017)"]
parameter_bounds["m"] = [0.8, 1, r"atm$\cdot$µm$^{-1}$", "Wang et al. (2017)", "Wang et al. (2012)"]
parameter_bounds["r"] = [0.05 * 10 ** (-12), 0.08 * 10 ** (-12),
                         r"dm$^3$ µm$^{-1}$", "Wang et al. (2017)", "Wang et al. (2012)"]
parameter_bounds["s"] = [0.1 * 10 ** (-12), 0.3 * 10 ** (-12), r"dm$^3$", "Wang et al. (2012)", "Wang et al. (2017)"]

For the apoplastic concentration, we use the value from the OnGuard model as the lower bound, and use a combination of values which have been collated by Roelfsema and Hedrich (2002) from Lohaus et al. (2001) for the upper bound:

In [ ]:
apoplastic_concs_wang_2017 = {
    "K": 10,
    "Ca": 1,
    "Cl": 12,
    "Suc": 0.01,
    "MH2": 3.2*10**(-6),
    "MH": 7.9*10**(-5),
    "M": 0.00999,
}
apoplastic_conc_wang_2017 = sum(apoplastic_concs_wang_2017.values())

In [ ]:
apoplastic_concs_roelfsema_2002 = {
    "K": 13,
    "Ca": 0.7,
    "Cl": 11,
    "Suc": 1.6,
    "Mal": 0.7,
    "Amino Acids": 9.6,
    "Hexoses": 0.7,
}
apoplastic_conc_roelfsema_2002 = sum(apoplastic_concs_roelfsema_2002.values())

In [ ]:
parameter_bounds["C_apo"] = [apoplastic_conc_wang_2017*10 **
                             (-3), apoplastic_conc_roelfsema_2002*10**(-3), r"mol$\cdot$dm$^{-3}$", "Wang et al. (2017)", "Roelfsema and Hedrich (2002)"]

## Apertures

For the closed aperture, we'll use 1 µm value from Jezek and Blatt (2017) as a lower bound and 4 µm from Wang et al. (2017) as an upper bound.

In [ ]:
parameter_bounds["A_closed"] = [1, 4, r"µm", "Jezek and Blatt (2017)", "Wang et al. (2017)"]

For the open aperture, we'll use the 2.75 µm from Horrer et al. (2016) as the lower bound and 12 µm from Wang et al. (2017) as the upper bound.

In [ ]:
parameter_bounds["A_open"] = [2.75, 12, r"µm", "Horrer et al. (2016)", "Wang et al. (2017)"]

For the proportion of photons that are absorbed by the leaf we can just go for 10% above and below the value that is reported by 

## ATPase

We'll go between constraining the ATPase to 0 and the level of ATPase that Flütsch et al. (2020) measured (17 fmoles$\cdot$GC$^{-1}\cdot$h$^{-1}$).

In [ ]:
parameter_bounds["ATPase"] = [
    0, 17, r"fmoles$\cdot$GC$^{-1}\cdot$h$^{-1}$", "Supplementary", "Flütsch et al. (2020)"]

In [ ]:
params_df = pd.DataFrame.from_dict(parameter_bounds, orient="index", columns=[
                                   "Lower", "Upper", "Units", "Source Lower", "Source Upper"])
params_df.to_csv("../outputs/constraint_scan/parameter_bounds.csv")
params_df.to_csv("../plant_cell_paper/supplemental_table_2.csv")

In [ ]:
params_df

,Lower,Upper,Units,Source Lower,Source Upper
P_abs,8.100000e-01,9.900000e-01,Dimensionless,"90% of Zhu, Long, and Ort (2010)","110% of Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,2.400000e-04,m,Wuyts et al. (2010),Ramonell et al. (2001)
A_l,1.000000e+00,1.000000e+00,m$^2$,Fixed,Fixed
V_gc_ind,4.750000e-13,4.100000e-12,dm$^3$,Jezek and Blatt (2017),Hills et al. (2012)
FqFm,7.900000e-01,9.000000e-01,Dimensionless,Lawson (2003),Lawson (2003)
R_ch,3.500000e-02,1.833333e-01,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)","Fujiwara, Sanjaya, and Itoh (2019)"
L_air,1.850000e-01,3.700000e-01,Dimensionless,Ramonell et al. (2001),Earles et al. (2018)
L_epidermis,1.000000e-01,2.400000e-01,Dimensionless,Willmer and Fricker (1996),Ramonell et al. (2001)
Vac_frac,7.510000e-01,9.000000e-01,Dimensionless,Wang et al. (2017),Andrés et al. (2014)
T,2.831500e+02,2.981500e+02,K,10C,25C


### Generate constraints array

In [ ]:
np.random.seed(31415)
lhs_df = pd.DataFrame(pyDOE.lhs(len(params_df), samples=1000), columns=params_df.index)

In [ ]:
lhs_df

,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,L_air,L_epidermis,Vac_frac,T,R,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase
0,0.598283,0.382096,0.360196,0.096058,0.430689,0.582219,0.965869,0.174251,0.310949,0.676081,0.350129,0.307338,0.329932,0.277458,0.840608,0.841386,0.201685,0.551405,0.514984,0.049579
1,0.395895,0.834124,0.403975,0.670988,0.555089,0.224124,0.882185,0.501864,0.275227,0.569901,0.082091,0.561464,0.694335,0.266096,0.071217,0.924072,0.281912,0.943401,0.576307,0.877863
2,0.344619,0.107536,0.511027,0.111285,0.243491,0.818683,0.350414,0.215672,0.200350,0.145697,0.429363,0.875809,0.939880,0.508252,0.183596,0.806334,0.224396,0.648303,0.356931,0.092135
3,0.751033,0.026151,0.655720,0.153535,0.469648,0.780116,0.611487,0.812458,0.290134,0.605398,0.299345,0.557861,0.151735,0.558696,0.037498,0.799401,0.308932,0.694694,0.995071,0.095047
4,0.350846,0.468450,0.565799,0.351265,0.358483,0.736410,0.850171,0.353026,0.487027,0.192733,0.525284,0.117137,0.556712,0.203088,0.819383,0.454055,0.891758,0.308953,0.885298,0.012697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.467396,0.523844,0.164472,0.926516,0.000909,0.586268,0.905711,0.412156,0.702914,0.323484,0.330463,0.400680,0.868469,0.693086,0.472039,0.804197,0.740728,0.702568,0.664242,0.072167
996,0.239250,0.608484,0.862220,0.038286,0.637539,0.423608,0.911187,0.865959,0.239153,0.688343,0.074856,0.380041,0.592303,0.853966,0.603645,0.718571,0.511484,0.780458,0.973929,0.979101
997,0.898836,0.490542,0.558371,0.524065,0.195645,0.396144,0.308697,0.359794,0.433892,0.938592,0.210794,0.874929,0.802742,0.793727,0.547461,0.972227,0.151030,0.168309,0.122978,0.333262
998,0.130157,0.051106,0.018765,0.517490,0.432583,0.866775,0.125102,0.968265,0.278008,0.333859,0.928970,0.011777,0.761461,0.388600,0.938613,0.293328,0.662041,0.464024,0.512581,0.482024


In [ ]:
constraints_df = generate_constraints_df(lhs_df, params_df)
constraints_df.head()

Removing 28 combinations where open aperture is smaller than closed


,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,L_air,L_epidermis,Vac_frac,T,R,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase
0,0.917691,0.000197,1.0,8.232099e-13,0.837376,0.121362,0.363686,0.124395,0.797331,293.291212,0.08205,4.756498e+08,1.829932,0.855492,7.521823e-14,2.682773e-13,0.025900,2.654215,7.513602,0.842851
1,0.881261,0.000228,1.0,2.907333e-12,0.851060,0.068245,0.348204,0.170261,0.792009,291.698522,0.08205,7.267261e+08,2.194335,0.853219,5.213651e-14,2.848145e-13,0.027046,3.830202,8.080839,14.923673
2,0.872031,0.000178,1.0,8.784094e-13,0.816784,0.156438,0.249827,0.130194,0.780852,285.335460,0.08205,1.037300e+09,2.439880,0.901650,5.550787e-14,2.612669e-13,0.026224,2.944908,6.051611,1.566297
3,0.945186,0.000172,1.0,1.031566e-12,0.841661,0.150717,0.298125,0.213744,0.794230,292.230971,0.08205,7.231671e+08,1.651735,0.911739,5.112495e-14,2.598802e-13,0.027432,3.084081,11.954409,1.615792
4,0.873152,0.000203,1.0,1.748335e-12,0.829433,0.144234,0.342282,0.149424,0.823567,286.040989,0.08205,2.877316e+08,2.056712,0.840618,7.458150e-14,1.908111e-13,0.035754,1.926858,10.939009,0.215853


In [ ]:
constraints_df.to_csv("../outputs/constraint_scan/constraints_df.csv")